In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import matplotlib.animation as animation
from IPython.display import HTML
import seaborn as sns
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import folium

## Load predictions

In [ ]:
data2 = np.load('data/dcrnn_denhaag_S36_DR2_RNN2_predictions.npz')
data1 = np.load('data/dcrnn_denhaag_S36_predictions.npz')

with open('data/sensor_graph/graph_sensor_ids.txt', 'r') as file:
    content = file.read()
    native_indices = [int(index.strip()) for index in content.split(',')]


# Load the data
data_path = 'data/DH.h5'
df = pd.read_hdf(data_path)

# Extract the timestamps for the last 572 example (Test set)
timestamps = df.index[-572:]
timestamps_str = [ts.strftime('%H-%M\n%d/%m/%Y') if ts.strftime('%H-%M') == '00-00' or ts.strftime('%H-%M') == '12-00' else ts.strftime('%H-%M') for ts in timestamps]
samples_per_day=96
samples_per_hour = samples_per_day // 24

# Extract the ground truth and predictions for the last 572 examples (Test set)
prediction2 = data2['prediction']
prediction1 = data1['prediction']
truth = data1['truth']

horizon = truth.shape[0]

# Define the horizon labels for 15 min intervals in the format [15min, 30min, 45min, 1h, 1h15min, 1h30min, 1h45min, 2h]:
horizon_labels = ['15min', '30min', '45min', '1h', '1h15min', '1h30min', '1h45min', '2h']




## Define the error metrics

In [ ]:
def errors(truth, prediction, n, sensor):
    ae = []
    se = []
    pe = []
    # Iterate over the time
    avg_truth = np.abs(np.mean(truth[n,:,sensor]))
    for i in range(truth.shape[1]): 
        # Append absolute error
        ae.append(np.abs(truth[n,i,sensor] - prediction[n,i,sensor]))
        # Append squared error
        se.append(np.square(truth[n,i,sensor] - prediction[n,i,sensor]))
        # Append absolute percentage error
        pe.append(np.abs((truth[n,i,sensor] - prediction[n,i,sensor]) / truth[n,i,sensor])*100 if truth[n,i,sensor] > 0 else np.abs((truth[n,i,sensor] - prediction[n,i,sensor]) / avg_truth)*100)
    mae = np.mean(ae)
    mae_std = np.std(ae)
    rmse = np.sqrt(np.mean(se))
    rmse_std = np.sqrt(np.std(se))
    mape = np.mean(pe)
    mape_std = np.std(pe)
    
    return mae, mae_std, rmse, rmse_std, mape, mape_std

## Compute errors for each sensor and each horizon for both models

In [ ]:
# Model 1: NoConv
errorNoConv = np.empty((horizon, truth.shape[2],6))

for n in range(horizon):
    for s in range(truth.shape[2]):
        errorNoConv[n,s,0], errorNoConv[n,s,1],errorNoConv[n,s,2], errorNoConv[n,s,3], errorNoConv[n,s,4], errorNoConv[n,s,5] = errors(truth, prediction1, n, s)

# Error is a 3D array with shape (horizon, num_sensors, 6)
average_error_NoConv = np.empty((horizon,3,2))

average_error_NoConv[:,0,0] = np.mean(errorNoConv[:,:,0], axis=1)  # MAE
average_error_NoConv[:,0,1] = np.std(errorNoConv[:,:,0], axis=1)   # MAE std
average_error_NoConv[:,1,0] = np.mean(errorNoConv[:,:,2], axis=1)  # RMSE
average_error_NoConv[:,1,1] = np.std(errorNoConv[:,:,2], axis=1)   # RMSE std
average_error_NoConv[:,2,0] = np.mean(errorNoConv[:,:,4], axis=1)  # MAPE
average_error_NoConv[:,2,1] = np.std(errorNoConv[:,:,4], axis=1)   # MAPE std

# Model 2: DCRNN
errorDCRNN = np.empty((horizon, truth.shape[2],6))

for n in range(horizon):
    for s in range(truth.shape[2]):
        errorDCRNN[n,s,0], errorDCRNN[n,s,1], errorDCRNN[n,s,2], errorDCRNN[n,s,3], errorDCRNN[n,s,4], errorDCRNN[n,s,5] = errors(truth, prediction2, n, s)
        
#  Error matrix is of shape (horizon, num_sensors, 6)
average_error_DCRNN = np.empty((horizon, 3, 2))  # Initialize average_error array


# Compute means and standard deviations of error metrics 
average_error_DCRNN[:,0,0] = np.mean(errorDCRNN[:,:,0], axis=1)  # MAE
average_error_DCRNN[:,0,1] = np.std(errorDCRNN[:,:,0], axis=1)  # MAE_std
average_error_DCRNN[:,1,0] = np.mean(errorDCRNN[:,:,2], axis=1)  # RMSE
average_error_DCRNN[:,1,1] = np.std(errorDCRNN[:,:,2], axis=1)  # RMSE_std
average_error_DCRNN[:,2,0] = np.mean(errorDCRNN[:,:,4], axis=1)  # MAPE
average_error_DCRNN[:,2,1] = np.std(errorDCRNN[:,:,4], axis=1) # MAPE_std

# Extract MAE, RMSE and MAPE values from average_error arrays
mae_values_DCRNN = errorDCRNN[:,:,0].T  # MAE values are at index 0
mae_values_NoConv = errorNoConv[:,:,0].T  # MAE values are at index 0
rmse_values_DCRNN = errorDCRNN[:,:,2].T  # RMSE values are at index 2
rmse_values_NoConv = errorNoConv[:,:,2].T  # RMSE values are at index 2
mape_values_DCRNN = errorDCRNN[:,:,4].T  # MAPE values are at index 4
mape_values_NoConv = errorNoConv[:,:,4].T  # MAPE values are at index 4

In [ ]:
# Print the average MAE, RMSE and MAPE values with their standard deviations for each horizon:
print('Average MAE values for each horizon:')
print('NoConv: ', ["{:.1f} (±{:.1f})".format(mean, std) for mean, std in zip(average_error_NoConv[:,0,0], average_error_NoConv[:,0,1])])
print('DCRNN: ', ["{:.1f} (±{:.1f})".format(mean, std) for mean, std in zip(average_error_DCRNN[:,0,0], average_error_DCRNN[:,0,1])])
print('\nAverage RMSE values for each horizon:')
print('NoConv: ', ["{:.1f} (±{:.1f})".format(mean, std) for mean, std in zip(average_error_NoConv[:,1,0], average_error_NoConv[:,1,1])])
print('DCRNN: ', ["{:.1f} (±{:.1f})".format(mean, std) for mean, std in zip(average_error_DCRNN[:,1,0], average_error_DCRNN[:,1,1])])
print('\nAverage MAPE values for each horizon:')
print('NoConv: ', ["{:.1f}% (±{:.1f}%)".format(mean, std) for mean, std in zip(average_error_NoConv[:,2,0], average_error_NoConv[:,2,1])])
print('DCRNN: ', ["{:.1f}% (±{:.1f}%)".format(mean, std) for mean, std in zip(average_error_DCRNN[:,2,0], average_error_DCRNN[:,2,1])])


## Plot the error metrics as a function of the prediction horizon

In [ ]:
# Plot the error metrics as a function of the prediction horizon

# MAE
fig, ax = plt.subplots(3,1, figsize=(10,10))
ax[0].plot(average_error_NoConv[:,0,0], label='MAE_NoConv', color='gray')
ax[0].fill_between(np.arange(horizon), average_error_NoConv[:,0,0] - average_error_NoConv[:,0,1], average_error_NoConv[:,0,0] + average_error_NoConv[:,0,1], alpha=0.2, color='gray')
ax[0].plot(average_error_DCRNN[:,0,0], label='MAE_DCRNN')
ax[0].fill_between(np.arange(horizon), average_error_DCRNN[:,0,0] - average_error_DCRNN[:,0,1], average_error_DCRNN[:,0,0] + average_error_DCRNN[:,0,1], alpha=0.3)
ax[0].set_ylabel('MAE')
ax[0].set_xlabel('Prediction Horizon')
ax[0].set_xticks(np.arange(0,horizon))
ax[0].set_xticklabels(horizon_labels)
# ax[0].set_ylim(0,20)
ax[0].legend()
ax[0].set_title('Mean Absolute Error')

# RMSE
ax[1].plot(average_error_NoConv[:,1,0], label='RMSE_NoConv', color='gray')
ax[1].fill_between(np.arange(horizon), average_error_NoConv[:,1,0] - average_error_NoConv[:,1,1], average_error_NoConv[:,1,0] + average_error_NoConv[:,1,1], alpha=0.2, color='gray')
ax[1].plot(average_error_DCRNN[:,1,0], label='RMSE_DCRNN')
ax[1].fill_between(np.arange(horizon), average_error_DCRNN[:,1,0] - average_error_DCRNN[:,1,1], average_error_DCRNN[:,1,0] + average_error_DCRNN[:,1,1], alpha=0.3)
ax[1].set_ylabel('RMSE')
ax[1].set_xlabel('Prediction Horizon')
ax[1].set_xticks(np.arange(0,horizon))
ax[1].set_xticklabels(horizon_labels)
# ax[1].set_ylim(0,20)
ax[1].legend()
ax[1].set_title('Root Mean Squared Error')

# MAPE
ax[2].plot(average_error_NoConv[:,2,0], label='MAPE_NoConv', color='gray')
ax[2].fill_between(np.arange(horizon), average_error_NoConv[:,2,0] - average_error_NoConv[:,2,1], average_error_NoConv[:,2,0] + average_error_NoConv[:,2,1], alpha=0.2, color='gray')
ax[2].plot(average_error_DCRNN[:,2,0], label='MAPE_DCRNN')
ax[2].fill_between(np.arange(horizon), average_error_DCRNN[:,2,0] - average_error_DCRNN[:,2,1], average_error_DCRNN[:,2,0] + average_error_DCRNN[:,2,1], alpha=0.3)
ax[2].set_ylabel('MAPE')
ax[2].set_xlabel('Prediction Horizon')
ax[2].set_xticks(np.arange(0,horizon))
ax[2].set_xticklabels(horizon_labels)
# ax[2].set_ylim(0,50)
ax[2].legend()
ax[2].set_title('Masked Mean Absolute Percentage Error')

plt.tight_layout()



## Plot MAE with standard deviation for selected horizon

In [ ]:
# Error matrix is of shape (horizon, num_sensors, 6)
num_sensors = errorDCRNN.shape[1]
mae_values = errorDCRNN[:,:,0]  # MAE values are at index 0
mae_std_values = errorDCRNN[:,:,0]  # MAE std values are at index 1

future = 7
# Plot the MAE values for each sensor for selected horizon:
fig, ax = plt.subplots(figsize=(20, 10))
ax.set_title('MAE_DCRNN values for each sensor for horizon ' + str(horizon_labels[future]))
ax.set_xlabel('Sensor')
ax.set_ylabel('MAE')
ax.set_xticks(np.arange(num_sensors))
ax.set_xticklabels(native_indices, rotation=90)
ax.bar(np.arange(num_sensors), mae_values[future,:], yerr=mae_std_values[future,:], align='center', alpha=0.5, ecolor='black', capsize=10)
# ax.set_ylim([0, 0.5])
ax.set_xlim([-1, num_sensors])
ax.yaxis.grid(True)



## Plot progression of error metrics as a function of the prediction horizon

In [ ]:
# Create a new figure with 3 subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 25))

# Horizon labels
horizon_labels = ['15min', '30min', '45min', '1h', '1h15min', '1h30min', '1h45min', '2h']  # Adjust this list to match your horizon count

# Define the color normalization to fit all heatmaps

# Range of values using all sets
# vmin = min(np.min(mae_values_DCRNN), np.min(rmse_values_DCRNN), np.min(mape_values_DCRNN))
# vmax = max(np.max(mae_values_DCRNN), np.max(rmse_values_DCRNN), np.max(mape_values_DCRNN))

# Range of values using MAE for one horizon only
vmin = np.min(mae_values_DCRNN[:,future])
vmax = np.max(mae_values_DCRNN[:,future])

norm = colors.Normalize(vmin=vmin, vmax=vmax)
norm_mape = colors.Normalize(vmin=vmin/100, vmax=vmax/100)

# Create the MAE heatmap
sns.heatmap(mae_values_DCRNN, annot=True, fmt=".1f", yticklabels=native_indices , xticklabels=horizon_labels, cmap="RdYlGn_r", ax=axs[0], norm=norm, cbar=False)

# Configure the MAE plot
axs[0].set_title('MAE_DCRNN Heatmap')
axs[0].set_xlabel('Horizon')
axs[0].set_ylabel('Sensor ID')


# Create the RMSE heatmap
sns.heatmap(rmse_values_DCRNN, annot=True, fmt=".1f", yticklabels=False, xticklabels=horizon_labels, cmap="RdYlGn_r", ax=axs[1], cbar=False)

# Configure the RMSE plot
axs[1].set_title('RMSE_DCRNN Heatmap')
axs[1].set_xlabel('Horizon')



# Create the MAPE heatmap
# Use format to have percentage sign: fmt="%" or fmt=".1%" for 1 decimal and divide by 100: 
sns.heatmap(mape_values_DCRNN/100, annot=True, fmt="0.1%", yticklabels=False, xticklabels=horizon_labels, cmap="RdYlGn_r", ax=axs[2], cbar=False)

# Configure the MAPE plot
axs[2].set_title('MAPE_DCRNN Heatmap')
axs[2].set_xlabel('Horizon')


# Show the plot
plt.tight_layout()
plt.show()


## Create a map of the sensors with the MAE values color-coding for a selected horizon (future) [Nodes a clickable]

In [ ]:
# Load the sensor locations
locations_full = pd.read_csv('data\sensor_graph\denhaag_locations_type.csv', index_col='sensor_id')

# Extract the locations for the sensors in the dataset (native_indices)
locations = locations_full.loc[native_indices]

# Add MAE to the locations DataFrame
locations['mae'] = errorDCRNN[future,:,0]  # MAE values for horizon 7 are at index [7,:,0]
locations['mape'] = errorDCRNN[future,:,4]  # RMSE values for horizon 7 are at index [7,:,1]

# Create a map centered at the average location
m = folium.Map(location=locations[['latitude', 'longitude']].mean().to_list(), zoom_start=13, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
               attr='Esri')

# Create a color map
min_mae = locations['mae'].min()
max_mae = locations['mae'].max()
min_mape = locations['mape'].min()
max_mape = locations['mape'].max()
cmap = colormaps.get_cmap('RdYlGn_r')  # '_r' indicates a reversed colormap
norm = colors.Normalize(vmin=min_mae, vmax=max_mae)
norm_mape = colors.Normalize(vmin=min_mape, vmax=max_mape)

# Function to map MAE to color
def get_color(mae):
    return colors.to_hex(cmap(norm(mae)))

# Function to map MAPE to color
def get_color_mape(mape):
    return colors.to_hex(cmap(norm_mape(mape)))

# Add markers to the map
for idx, row in locations.iterrows():
    folium.CircleMarker(location=(row['latitude'], row['longitude']),
                        radius=7,
                        color=get_color_mape(row['mape']),
                        fill=True,
                        fill_opacity=1,
                        fill_color=get_color(row['mae']),
                        popup=folium.Popup(f'Sensor ID: {idx}<br>MAE: {row["mae"]:.1f} (Fill)<br>MAPE: {row["mape"]:.1f} (Stroke)' , max_width=300) # Use f-strings to format the popup, sensor_id is an integer and mae is a float: 
                       ).add_to(m)

# Show the map
m

## Create a heatmap for the differences in performance between the two models and show them on the map

In [ ]:
delta_mae = (mae_values_DCRNN - mae_values_NoConv)/mae_values_NoConv
delta_rmse = (rmse_values_DCRNN - rmse_values_NoConv)/rmse_values_NoConv
delta_mape = (mape_values_DCRNN - mape_values_NoConv)/mape_values_NoConv

# Range of values
max_delta_mae = np.max(np.abs(delta_mae))
max_delta_rmse = np.max(np.abs(delta_rmse))
max_delta_mape = np.max(np.abs(delta_mape))

min_delta_mae = -max_delta_mae
min_delta_rmse = -max_delta_rmse
min_delta_mape = -max_delta_mape


# Create a new figure with 4 subplots
fig, axs = plt.subplots(1, 3, figsize=(16, 25))

# Horizon labels
horizon_labels = ['15min', '30min', '45min', '1h', '1h15min', '1h30min', '1h45min', '2h']  # Adjust this list to match your horizon count

# Create the MAE_NoConv heatmap
sns.heatmap(delta_mae, annot=True, fmt=".1%", yticklabels=native_indices, xticklabels=horizon_labels, cmap="RdYlGn_r", vmin=min_delta_mae, vmax=max_delta_mae, ax=axs[0], cbar=False)

# Configure the RMSE plot
axs[0].set_title(r'$\Delta$MAE')
axs[0].set_xlabel('Horizon')
axs[0].set_ylabel('Sensor ID')

# Create the RMSE heatmap
sns.heatmap(delta_rmse, annot=True, fmt=".1%", yticklabels=False, xticklabels=horizon_labels, cmap="RdYlGn_r", vmin=min_delta_rmse, vmax=max_delta_rmse, ax=axs[1], cbar=False)

# Configure the RMSE plot
axs[1].set_title(r'$\Delta$RMSE')
axs[1].set_xlabel('Horizon')

# Create the MAPE heatmap
sns.heatmap(delta_mape, annot=True, fmt=".1%", yticklabels=False, xticklabels=horizon_labels, cmap="RdYlGn_r", vmin=min_delta_mape, vmax=max_delta_mape, ax=axs[2], cbar=False)

# Configure the MAPE plot
axs[2].set_title(r'$\Delta$MAPE')
axs[2].set_xlabel('Horizon')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Load the sensor locations
locations_full = pd.read_csv('data\sensor_graph\denhaag_locations_type.csv', index_col='sensor_id')

# Extract the locations for the sensors in the dataset (native_indices)
locations = locations_full.loc[native_indices]

# Add MAE to the locations DataFrame
locations['delta_mae'] = delta_mae[:, future]  # MAE values for horizon 7 are at index [7,:,0]
locations['delta_mape'] = delta_mape[:, future]  # MAE values for horizon 7 are at index [7,:,0]

# Create a map centered at the average location
m = folium.Map(location=locations[['latitude', 'longitude']].mean().to_list(), zoom_start=13, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
               attr='Esri')

# Create a color map
min_mae = locations['delta_mae'].min()
max_mae = locations['delta_mae'].max()
min_mape = locations['delta_mape'].min()
max_mape = locations['delta_mape'].max()
cmap = colormaps.get_cmap('RdYlGn_r')  # '_r' indicates a reversed colormap
norm = colors.Normalize(vmin=min_delta_mae, vmax=max_delta_mae)
norm_mape = colors.Normalize(vmin=min_delta_mape, vmax=max_delta_mape)

# Function to map MAE to color
def get_color(mae):
    return colors.to_hex(cmap(norm(mae)))

# Function to map MAPE to color
def get_color_mape(mape):
    return colors.to_hex(cmap(norm_mape(mape)))

# Add markers to the map
for idx, row in locations.iterrows():
    folium.CircleMarker(location=(row['latitude'], row['longitude']),
                        radius=7,
                        color=get_color_mape(row['delta_mape']),
                        fill=True,
                        fill_opacity=1,
                        fill_color=get_color(row['delta_mae']),
                        popup=folium.Popup(f'Sensor ID: {idx}<br>Delta MAE: {row["delta_mae"]:.1%} (Fill)<br>Delta MAPE: {row["delta_mape"]:.1%} (Stroke)' , max_width=300) # Use f-strings to format the popup, sensor_id is an integer and mae is a float: 
                       ).add_to(m)

# Show the map
m

## Animation: 24-hour window moving across the test set for a selected sensor_ID and horizon

In [ ]:

native_id = 702112  # Replace this with the native ID of the sensor you want to plot

horizon = 8 # horizon to plot

step = 2 # in hours between frames

window = 24 # in hours


#########


# Compute the number of frames to plot:
num_frames = int((truth.shape[1] - (window*samples_per_hour+4))/(samples_per_hour*step))-1 


if native_id in native_indices:
    node_index = native_indices.index(native_id)
else:
    print(f"Sensor {native_id} not found in native_indices.")
    node_index = None

# Calculate the maximum value over all frames, predictions and truth sets
max_value = max(np.max(truth[horizon-1, :, node_index]), 
                np.max(prediction1[horizon-1, :, node_index]),
                np.max(prediction2[horizon-1, :, node_index]))
# Add a small margin (10% of the max value)
max_value *= 1.1


fig, ax = plt.subplots(figsize=(12, 6))

# Function to update the plot for a given hour
def update(hour):
    start_sample = step*hour * samples_per_hour + 4
    end_sample = start_sample + window * samples_per_hour

    if node_index is not None:
        # prediction_node3 = prediction3[0, start_sample + shift:end_sample + shift, node_index]
        prediction_node2 = prediction2[horizon-1, start_sample:end_sample, node_index]
        prediction_node1 = prediction1[horizon-1, start_sample:end_sample, node_index]
        truth_node = truth[horizon-1, start_sample:end_sample, node_index]

        # Clear the current plot
        ax.clear()

        ax.plot(truth_node, label='Truth', color='tab:orange')
        ax.plot(prediction_node1, label='NoConv',color='tab:red')
        ax.plot(prediction_node2, label='DCRNN (DR2+RNN2)',color='tab:blue')
        # ax.plot(prediction_node3, label='DR2',color='tab:green')   

        ax.set_xlabel('Timestamp')
        ax.set_ylabel('Number of vehicles')
        ax.legend()
        ax.set_title(f'Sensor {native_id}')
        ax.set_ylim([0, max_value])  # Set the top y-limit to max_value
        ax.set_xticks(range(0, len(timestamps[start_sample:end_sample])+1, 2 * samples_per_hour))
        ax.set_xticklabels(timestamps_str[start_sample:end_sample+1:2 * samples_per_hour], rotation=0, ha='center')

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=range(0, num_frames), interval=100)
plt.close()
# Display the animation
HTML(ani.to_jshtml())


## Extras

In [ ]:
# ## Static plot for a single sensor [24 hours - window]

# # Select 24-hour window
# start_hour = 23  # Change this to select the starting hour of the 3-hour window
# start_sample = start_hour * samples_per_hour
# end_sample = start_sample + 24 * samples_per_hour

# # Select the sensor to display
# native_id = 702112

# if native_id in native_indices:
#     node_index = native_indices.index(native_id)
# else:
#     print(f"Sensor {native_id} not found in native_indices.")
#     node_index = None

# # Choose the horizon to display
# horizon = 8

# # prediction_node3 = prediction3[0, start_sample + shift:end_sample + shift, node_index]
# prediction_node2 = prediction2[horizon-1, start_sample:end_sample, node_index]
# prediction_node1 = prediction1[horizon-1, start_sample:end_sample, node_index]
# truth_node = truth[horizon-1, start_sample:end_sample, node_index]

# fig, ax = plt.subplots(figsize=(12, 6))
# ax.plot(truth_node, label='Truth', color='tab:orange')
# ax.plot(prediction_node1, label='NoConv',color='tab:red')
# ax.plot(prediction_node2, label='DCRNN (DR2+RNN2)',color='tab:blue')

# ax.set_xlabel('Timestamp')
# ax.set_ylabel('Number of vehicles')
# ax.legend()
# ax.set_title(f'Sensor {native_indices[node_index]}')
# ax.set_xticks(range(0, len(timestamps[start_sample:end_sample])+1, 2 * samples_per_hour))
# ax.set_xticklabels(timestamps_str[start_sample:end_sample+1:2 * samples_per_hour], rotation=0, ha='center')

# plt.show()

In [ ]:
### Static plot predictions for all sensors (123 subplots) [24 hours windows]

# # Select 24-hour window
# day = 0  # Change this to select the starting day of the 24-hour window

# start_hour = 24*day  # Change this to select the starting hour of the 24-hour window
# start_sample = start_hour * samples_per_hour
# end_sample = start_sample + 24 * samples_per_hour

# fig, axs = plt.subplots(62, 2, figsize=(20, 400))
# fig.tight_layout(pad=5.0)


# for i, native_index in enumerate(native_indices):
#     row = i // 2
#     col = i % 2

#     prediction_node2 = prediction2[0, start_sample:end_sample, i]
#     prediction_node1 = prediction1[0, start_sample:end_sample, i]
#     truth_node = truth[0, start_sample:end_sample, i]

#     axs[row, col].plot(truth_node, label='Truth', color='tab:orange')    
#     axs[row, col].plot(prediction_node1, label='DR0',color='tab:red')
#     axs[row, col].plot(prediction_node2, label='DR1',color='tab:blue')


#     axs[row, col].set_xlabel('Timestamp')
#     axs[row, col].set_ylabel('Number of vehicles')
#     axs[row, col].legend()
#     axs[row, col].set_title(f'Sensor {native_index}')
#     axs[row, col].set_xticks(range(0, len(timestamps[start_sample:end_sample+1]), 2*samples_per_hour))
#     axs[row, col].set_xticklabels(timestamps_str[start_sample:end_sample+1:2*samples_per_hour], rotation=0, ha='center')

# plt.show()